In [1]:
import os
import math
import time

In [8]:
path_to_TSP_folder = "heu_e_met_tsp_instances/EUC_2D"
files = [f for f in os.listdir(path_to_TSP_folder) if os.path.isfile(os.path.join(path_to_TSP_folder, f))]
print(files)

['pr124.tsp', 'kroA150.tsp', 'kroA100.tsp', 'rat195.tsp', 'kroD100.tsp', 'pr152.tsp', 'kroB150.tsp', 'pr136.tsp', 'kroE100.tsp', 'pr107.tsp', 'pr76.tsp', 'kroC100.tsp', 'kroB100.tsp', 'lin105.tsp', 'kroB200.tsp', 'berlin52.tsp', 'pr144.tsp', 'rat99.tsp', 'kroA200.tsp', 'st70.tsp']


In [3]:
class City:
    def __init__(self, index: int, x: float, y:float):
        """x and y are the coordinates of the city"""
        self.index = int(index)
        self.x = x
        self.y = y

    def distance_to(self, d):
        return math.sqrt((self.x - d.x)**2 + (self.y - d.y) ** 2)
    def __str__(self):
        return f"{self.index}, {self.x}, {self.y}"




# Heurística Construtiva

Dada uma instância do problema TSP, a heurística implementada procura sempre pelo vizinho mais próximo da cidade atual.

In [13]:


def constructive_heuristic(instance:list[City]):
    start = time.time()
    curr = instance[1]
    total_dist = 0
    visited = [False for _ in range(len(instance))]
    
    visited[0] = True
    visited[curr.index] = True
    start = time.time()
    path = [curr]
    while False in visited:
        

        next_city = None
        min_dist = math.inf
        
        for inst in instance[1:]:
            if not visited[inst.index]:
                
                d = curr.distance_to(inst)


                if d < min_dist:
                    min_dist = d
                    next_city = inst
        
        curr = next_city
        
        total_dist += round(min_dist)
        visited[curr.index] = True
        path.append(curr)
    
    total_time = time.time() - start
    path.append(instance[1])

    return path, total_dist + curr.distance_to(instance[1]), total_time

In [ ]:

TSP_instance = [None]
NUMBER_OF_EXECUTIONS = 30
instance_name = ""
for f in files:
    with open(os.path.join(path_to_TSP_folder, f), "r") as f:
        instance_name = f.readline().split(":")[1].strip()
        
        for line in f.readlines()[5:]:
            if(line.strip() == "EOF"):
                break

            values = [float(v.strip()) for v in line.split(" ") if v != "" and v!="\n"]# we will have [city number, x, y]

            if len(values) < 3:
                print("Error reading TSP instance")
            
            TSP_instance.append(City(*values))
        print(f"Current Instance: {instance_name.strip()}")
        print(f"Number of Cities: {len(TSP_instance) - 1}")
        t = 0
        cost = 0.0
        for _ in range(NUMBER_OF_EXECUTIONS):
            result, c , t1 = constructive_heuristic(TSP_instance)
            cost += c
            t += t1
            

        print(f"Cost: {cost / NUMBER_OF_EXECUTIONS:.02f}\nTime: {t / NUMBER_OF_EXECUTIONS * 1000:.4f} ms\nPath:" + ", ".join(str(c.index) for c in result))
        print("=======================================================")
        TSP_instance.clear()
        TSP_instance.append(None)

Current Instance: pr124
Number of Cities: 124
Cost: 69296.95
Time: 2.0490 ms
Path:1, 8, 2, 3, 4, 6, 32, 31, 30, 34, 33, 59, 61, 60, 62, 66, 65, 64, 63, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 92, 93, 94, 95, 96, 97, 98, 99, 100, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 82, 84, 81, 83, 67, 58, 57, 56, 55, 37, 38, 24, 25, 26, 39, 23, 22, 40, 21, 20, 19, 18, 45, 44, 43, 42, 41, 54, 52, 53, 46, 47, 48, 49, 50, 51, 15, 14, 11, 12, 13, 10, 9, 16, 17, 88, 89, 103, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 102, 90, 86, 87, 85, 91, 101, 36, 35, 29, 28, 27, 7, 5, 1
Current Instance: kroA150
Number of Cities: 150
Cost: 33611.74
Time: 3.1979 ms
Path:1, 130, 92, 8, 42, 122, 80, 31, 89, 133, 138, 148, 142, 105, 67, 108, 58, 28, 93, 131, 47, 109, 91, 98, 23, 45, 32, 11, 15, 17, 141, 59, 74, 21, 72, 113, 10, 84, 36, 38, 24, 18, 137, 79, 106, 90, 49, 6, 63, 75, 19, 53, 134, 88, 16, 22, 94, 70, 66, 65, 4, 97, 143, 56, 139, 119, 118, 124, 26, 129, 104, 102, 111, 99, 127,

# Heurística VND


In [ ]:
import random

def calculate_distance(tour:list[City]):
    cost = 0
    for i in range(len(tour) - 1):
        cost += tour[i].distance_to(tour[i + 1])
    return cost + tour[len(tour) - 1].distance_to(tour[0])

In [ ]:
def swap_neighborhood(tour:list[City]):
    neighbors = []
    for i in range(len(tour)):
        for j in range(i + 1, len(tour)):
            neighbor = tour[:]
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighbors.append(neighbor)
    return neighbors

def two_opt_neighborhood(tour:list[City]):
    neighbors = []
    for i in range(len(tour)):
        for j in range(i + 1, len(tour)):
            neighbor = tour[:i] + tour[i:j + 1][::-1] + tour[j + 1:]
            neighbors.append(neighbor)
    return neighbors

def insert_neighborhood(tour:list[City]):
    neighbors = []
    for i in range(len(tour)):
        for j in range(len(tour)):
            if i != j:
                neighbor = tour[:]
                city = neighbor.pop(i)
                neighbor.insert(j, city)
                neighbors.append(neighbor)
    return neighbors

In [ ]:
def variable_neighborhood_descent(cities:list[City]):

    current_tour = cities[1:].copy()
    random.shuffle(current_tour)
    random.shuffle(current_tour)
    current_distance = calculate_distance(current_tour)

    neighborhoods = [swap_neighborhood, two_opt_neighborhood, insert_neighborhood]
    
    improvement = True
    start = time.time()
    while improvement:
        improvement = False
        for neighborhood in neighborhoods:
            hasNext = True
            while hasNext:
                best_neighbor = None
                best_distance = current_distance
                hasNext = False
                for neighbor in neighborhood(current_tour):
                    neighbor_distance = calculate_distance(neighbor)
                    if neighbor_distance < best_distance:
                        best_neighbor = neighbor
                        best_distance = neighbor_distance
                        hasNext=True
                    
                if best_neighbor:
                    current_tour = best_neighbor
                    current_distance = best_distance
                    improvement = True
                    #print("Current distance:", current_distance)
                    break  # Restart from the first neighborhood
    
    
    return current_tour, current_distance, time.time() - start

In [7]:
TSP_instance = [None]
NUMBER_OF_EXECUTIONS = 1
instance_name = ""
for f in files:
    with open(os.path.join(path_to_TSP_folder, f), "r") as f:
        instance_name = f.readline().split(":")[1].strip()
        
        for line in f.readlines()[5:]:
            if(line.strip() == "EOF"):
                break

            values = [float(v.strip()) for v in line.split(" ") if v != "" and v!="\n"]# we will have [city number, x, y]

            if len(values) < 3:
                print("Error reading TSP instance")
            
            TSP_instance.append(City(*values))
        print(f"Current Instance: {instance_name.strip()}")
        print(f"Number of Cities: {len(TSP_instance) - 1}")
        t = 0
        cost = 0.0
        for i in range(NUMBER_OF_EXECUTIONS):
            #print("Iteration",i)
            result, c , t1 = variable_neighborhood_descent(TSP_instance)
            cost += c
            t += t1
            

        print(f"Cost: {cost / NUMBER_OF_EXECUTIONS:.02f}\nTime: {t / NUMBER_OF_EXECUTIONS * 1000:.4f} ms\nPath:" + ", ".join(str(c.index) for c in result))
        print("=======================================================")
        TSP_instance.clear()
        TSP_instance.append(None)

Current Instance: pr124
Number of Cities: 124
Cost: 59599.14
Time: 41756.1362 ms
Path:59, 36, 35, 29, 28, 27, 7, 8, 1, 2, 3, 4, 5, 6, 32, 31, 30, 34, 33, 62, 66, 65, 63, 64, 68, 69, 70, 71, 72, 73, 74, 75, 67, 83, 84, 82, 81, 76, 77, 78, 79, 80, 92, 93, 94, 95, 96, 97, 98, 99, 100, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 101, 91, 102, 103, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 89, 88, 87, 86, 90, 85, 54, 53, 52, 46, 47, 48, 49, 50, 51, 15, 14, 13, 12, 11, 10, 9, 16, 17, 18, 19, 45, 44, 43, 42, 41, 20, 21, 40, 39, 23, 22, 25, 24, 26, 38, 37, 55, 56, 57, 58, 60, 61
Current Instance: kroA150
Number of Cities: 150
Cost: 28302.76
Time: 85415.1297 ms
Path:1, 63, 6, 49, 113, 72, 21, 74, 141, 59, 127, 36, 99, 102, 111, 104, 38, 24, 18, 84, 10, 90, 106, 79, 137, 134, 53, 88, 16, 94, 22, 70, 66, 129, 26, 124, 118, 65, 4, 19, 75, 92, 8, 42, 122, 139, 97, 119, 143, 56, 80, 31, 89, 133, 138, 54, 40, 64, 144, 2, 44, 114, 73, 69, 108, 58, 61, 51, 125, 87, 145, 25, 81, 68, 8